# 本プログラムについて

## 目標
24時間後の電力のスポット価格を予測し、次の日の最安値で電力を買い、最高値で電力を売却する。

## 背景
近年太陽光発電の影響もあり、晴れの日の日中に電力価格が下落し、夜間に電力価格が高騰するケースがある。また、家庭での太陽光発電と蓄電池/電気自動車のバッテリーを組み合わせることで、微々たる量ではあるが、株の売買のように電力価格差で利益を得ることができる。

## 利用データ
各データを事前にExcelのPower Queryで処理し、CSVファイルに変換したものをPandasで読み込む。太陽光発電が盛んであり、日中と夜間の価格差が比較的大きい九州のデータを利用する。天候データは九州であればどこでもよい。今回は、下記サイトより太陽光発電所が多く存在する福岡県飯塚市の天候データを利用する。  
http://agora.ex.nii.ac.jp/earthquake/201103-eastjapan/energy/electrical-japan/type/8.html.ja  

###  
*   九州地区の電力スポット価格 (http://www.jepx.org/market/)
*   福岡県飯塚市の天候データ (https://www.data.jma.go.jp/obd/stats/etrn/select/prefecture.php?prec_no=82&block_no=&year=&month=&day=&view=)  

## 参考にした外部プログラム
https://qiita.com/sloth-hobby/items/8d4c7c1185d660921b7a

### 参考プログラムからの変更点
* 参考プログラムでは、入力1, 出力1であったが、本プログラムでは入力8, 出力1としている。それに伴い、プログラムを複数個所改変した。
* MakeDatasetクラスのmake_dataset関数で適切にデータセットが作成できるように元プログラムから大きく改変した。

## 独自の工夫点
### LSTMを採用した理由
時系列データを扱うことができるRNN、GRU、LSTMの中でLSTMが一番予測精度が高いと考え採用した。Time層が1000以上あるため、RNNでは勾配消失が起こりやすいのではと考える。

### BiLSTMを採用しない理由
BiLSTMでは未来の情報も利用して予測を行うため、今回のような純粋な時系列データの予測では不適当であると考える。未来のデータを利用ということは、予測するべきデータがモデル作成に利用されてしまうため、モデルがTrainデータに過剰に適合すると考える。

### パラメータ設定理由
* Time大きさ: 本来は季節要因による価格変動を学習させたかったため、1年分、つまり24時間x365日で設定したかったが、Google Clabがクラッシュするため、1008とした。
* 隠れ層の大きさ: LSTM層が1層であるため、表現力を上げるために大きめに設定している。
* LSTM層: LSTM層が1層、3層、5層と試したが1層が一番よい結果であった。
* Optimizer: 特に検証をしていないが、一番よいと言われているAdamを採用した。
* min batch size: なるべく小さくしてモデルの精度を上げられるようにした。メモリ不足により他の多くの箇所でクラッシュしていたため、mini batch sizeを大きくしてもよかったと考える。
* epoch: 6あたりで収束しているため、10で設定した。

# ライブラリインポート

In [ ]:
# pytorchのバージョンを固定
import sys
Google colab環境であるか判定
if 'google.colab' in sys.modules:
    print("Google Colab")
    # ライブラリのインストール
    !pip install nltk==3.6.5 torch==1.10.2 torchvision==0.11.3
else:
    print("Not Google Colab")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime
import pandas.tseries.offsets as offsets # pandasで日付を操作する

# データインポート

In [ ]:
import sys
# Google colab環境であるか判定
if 'google.colab' in sys.modules:
    # マウントを行う
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Not Google Colab")

Mounted at /content/drive


In [ ]:
# Google colab環境であるか判定
if 'google.colab' in sys.modules:
    data_path = '/content/drive/MyDrive/python/02_電力売買予測/Programs/Data.xlsx'
else:
    data_path = 'Data2.xlsx'

データインポートについて参照することhttps://pystyle.info/pytorch-how-to-create-custom-dataset-class/  
Datasetクラスについてhttps://qiita.com/typecprint/items/2d616b502dd063ba4e27

# データセット作成用クラス

In [ ]:
class MakeDataset(Dataset):
  def __init__(self, data_path, sequence_length):
    self.df = pd.read_excel(data_path)
    # 欠損をすぐ上の値で埋める
    self.df = self.df.fillna(method='ffill')
    print(self.df.isnull().sum())
    self.inputs = self.df.drop(['Date'], axis=1).to_numpy() # 説明変数+目的変数
    # self.inputs = df.filter(items=['Temp']).to_numpy()
    self.labels = self.df.filter(items=['Price']).to_numpy() # 目的変数
    self.date_hour = self.df.filter(items=['Date']).to_numpy() # 日時保存用
    self.sequence_length = sequence_length
    self.dataset_num = len(self.df)-sequence_length

  def make_dataset(self):
    dataset_inputs = []
    dataset_labels = []
    dataset_times = []
    for t in range(self.dataset_num):
      dataset_inputs.append([self.inputs[t + i] for i in range(self.sequence_length)])
      dataset_labels.append(self.labels[t + self.sequence_length])
      dataset_times.append(self.date_hour[t + self.sequence_length])
    print("data shapes => input: {}, output: {}, num. of data{}".format(np.array(dataset_inputs).shape,  np.array(dataset_labels).shape, np.array(dataset_times).shape))
    return np.array(dataset_inputs),  np.array(dataset_labels), np.array(dataset_times)


# モデル作成クラス

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, 
                #  num_layers, 
                 batch_first):
        super().__init__()
        self.rnn = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            # num_layers = num_layers,
                            batch_first = batch_first)
        self.output_layer = nn.Linear(hidden_size, output_size)

        # 下記の初期化の参考: https://pystyle.info/pytorch-parameters-initialization/
        nn.init.xavier_normal_(self.rnn.weight_ih_l0) #重みの初期化(ih)
        nn.init.orthogonal_(self.rnn.weight_hh_l0)# 重みの初期化(hh)

    def forward(self, inputs): #inputs: (バッチサイズ，時系列のサイズ，入力のサイズ)
        h, _= self.rnn(inputs) # rnnの最初の出力h: (バッチサイズ，時系列のサイズ，隠れ層のベクトルのサイズ)
        output = self.output_layer(h[:, -1]) # h[:, -1](バッチサイズ，隠れ層のベクトルのサイズ)

        return output

# Train Classの定義

In [ ]:
class Train():
    def __init__(self, input_size, output_size, hidden_size, 
                #  num_layers, 
                 batch_first, lr):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #使用可能であればcudaを使う
        print("device：", self.device)

        # modelのインスタンス化
        self.net = LSTM(input_size, output_size, hidden_size, 
                        # num_layers, 
                        batch_first).to(self.device)

        # MSEをロス計算に利用
        self.criterion = nn.MSELoss()

        # 最適化はAdam
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr, betas=(0.9, 0.999), amsgrad=True)

    # trainデータの入力と正解ラベルを使ってネットワークのパラメータを更新する処理
    def train_step(self, 
                   inputs, # 形式(バッチサイズ，時系列のサイズ，入力のサイズ)
                   labels # 形式(バッチサイズ，出力のサイズ)
                   ):
        inputs = torch.Tensor(inputs).to(self.device)
        labels = torch.Tensor(labels).to(self.device)
        self.net.train() #学習モード
        preds = self.net(inputs) #データをモデルに入れる
        loss = self.criterion(preds, labels)  #損失計算
        self.optimizer.zero_grad() #勾配の初期化
        loss.backward() #誤差逆伝播
        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
        # nn.utils.clip_grad_value_(self.net.parameters(), clip_value=2.0)
        self.optimizer.step() #パラメータ更新

        return loss, preds

    # testデータの入力と正解ラベルを使ってlossを計算
    def test_step(self, inputs, labels):
        inputs = torch.Tensor(inputs).to(self.device)
        labels = torch.Tensor(labels).to(self.device)
        self.net.eval() # 検証モード
        preds = self.net(inputs) #モデルにデータを入れる
        loss = self.criterion(preds, labels) #損失計算

        return loss, preds

    # 主にtrainデータのバッチを作る処理とlossを計算する処理を行っています．
    def train(self, train_inputs, train_labels, test_inputs, test_labels, epochs, batch_size, sequence_length, input_size):
        # torch.backends.cudnn.benchmark = True   # ネットワークがある程度固定であれば、高速化させる

        n_batches_train = int(train_inputs.shape[0] / batch_size)
        n_batches_test = int(test_inputs.shape[0] / batch_size)
        for epoch in range(epochs):
            print('-------------')
            print('Epoch {}/{}'.format(epoch+1, epochs))
            print('-------------')
            train_loss = 0.
            test_loss = 0.

            # make_dataset関数で作った順番通りに並んでいる配列の組み合わせをランダムに入れ替える
            train_inputs_shuffle, train_labels_shuffle = shuffle(train_inputs, train_labels)

            for batch in range(n_batches_train):
                start = batch * batch_size
                end = start + batch_size
                loss, _ = self.train_step(np.array(train_inputs_shuffle[start:end]), np.array(train_labels_shuffle[start:end]))
                train_loss += loss.item()
           
            for batch in range(n_batches_test):
                start = batch * batch_size
                end = start + batch_size
                loss, _ = self.test_step(np.array(test_inputs[start:end]), np.array(test_labels[start:end]))
                test_loss += loss.item()

            train_loss /= float(n_batches_train)
            test_loss /= float(n_batches_test)
            print('loss: {:.3}, test_loss: {:.3}'.format(train_loss, test_loss))

    # 学習を終えたモデルを使って予測を行い，それを正解ラベルと比較するため可視化する
    def pred_result_plt(self, test_inputs, test_labels, test_times, sequence_length, input_size, pred_step_list):
      # pred_step_list: 1~12の未来を予測するために、[0,1, ... 12]の配列を引数として渡す。
        print('-------------')
        print("start predict test!!")
        print("RMSE Ratio: 1つ先のtimeの予測結果と比較して、どれぐらいRMSEが悪化したかを測る指標")
        self.net.eval()
        # 予測した各出力と各RMSEを保存するためのリストを用意
        preds = [[] for num in range(len(pred_step_list))]
        rmses = [0.0 for num in range(len(pred_step_list))]

        #予測の出力を保存する処理
        for i in range(len(pred_step_list)): #iの最大値は予測したい未来の数
            for j in range(0, len(test_inputs), pred_step_list[i]): 

                input = np.array(test_inputs[j]).reshape(-1, sequence_length, input_size)
                for k in range(pred_step_list[i]): #kの最大値はiの時の予測幅
                    if j + k < test_labels.shape[0]:
                        input_tensor = torch.Tensor(input).to(self.device)
                        pred = self.net(input_tensor).data.cpu().numpy()
                        input = np.delete(input, 0, axis=1) #入力データの先頭を消去
                        input = np.insert(input, 2, pred[0][0], axis=1) #入力データの末尾に予測値を追加
                        preds[i].append(pred[0][0])
            preds[i] = np.array(preds[i]).reshape(-1)
            test_labels = test_labels.reshape(-1)
            rmses[i] = np.sqrt(np.sum(np.power((test_labels - preds[i]), 2)) / float(test_labels.shape[0]))
            print("pred_step = {}, rmse = {}, rmse_ratio = {}".format(i + 1, rmses[i], (rmses[i] - rmses[0]) / rmses[0]))

        #以下グラフ描画
        # plt_legend_list = ['label']
        # for i in range(len(pred_step_list)):
        #     plt.plot(test_times, preds[i])
        #     plt_legend_list.append('pred_' + str(pred_step_list[i]))
        # plt.plot(test_times, test_labels, c='#00ff00')
        # plt.xlabel('t')
        # plt.ylabel('y')
        # plt.legend(plt_legend_list)
        # plt.title('compare label and preds')
        # plt.show()
        return np.array(preds), np.array(rmses)

    

# TrainデータTestデータの準備

In [ ]:
sequence_length = 1008 #Google ColabのRAMが許される限界付近を指定
dataset = MakeDataset(data_path, sequence_length) # dataset作成
dataset_inputs, dataset_labels, dataset_times = dataset.make_dataset()

Date                0
Temp                0
SunshineDuration    0
Atm                 0
Rainfall            0
Weather             0
Humidity            0
Visibility          0
Price               0
dtype: int64
data shapes => input: (34056, 1008, 8), output: (34056, 1), num. of data(34056, 1)


# 学習前の初期設定

In [ ]:
# model prams
input_size = 8
output_size = 1
hidden_size = 128
batch_first = True

# train params
lr = 0.001
epochs = 10
batch_size = 4
test_size = 0.1
pred_step_list = [num for num in range(1, 13)] #次の日の予測を行いたいため、12時間を指定
num_layers = 1

# Trainインスタンス化
train = Train(input_size, output_size, hidden_size, 
              # num_layers, 
              batch_first, lr)

# datasetをTrain Testに分ける
train_inputs, test_inputs, train_labels, test_labels = train_test_split(dataset_inputs, dataset_labels, test_size=test_size, shuffle=False)
train_times, test_times = train_test_split(dataset_times, test_size=test_size, shuffle=False)

train.train(train_inputs, train_labels, test_inputs, test_labels, epochs, batch_size, sequence_length, input_size)
preds, rmses = train.pred_result_plt(test_inputs, test_labels, test_times, sequence_length, input_size, pred_step_list)

device： cuda
-------------
Epoch 1/10
-------------
loss: 36.1, test_loss: 97.7
-------------
Epoch 2/10
-------------
loss: 23.2, test_loss: 72.5
-------------
Epoch 3/10
-------------
loss: 18.6, test_loss: 62.6
-------------
Epoch 4/10
-------------
loss: 16.7, test_loss: 55.8
-------------
Epoch 5/10
-------------
loss: 15.7, test_loss: 53.1
-------------
Epoch 6/10
-------------
loss: 15.2, test_loss: 50.1
-------------
Epoch 7/10
-------------
loss: 15.0, test_loss: 49.7
-------------
Epoch 8/10
-------------
loss: 14.8, test_loss: 46.9
-------------
Epoch 9/10
-------------
loss: 14.6, test_loss: 50.1
-------------
Epoch 10/10
-------------
loss: 14.4, test_loss: 45.6
-------------
start predict test!!
RMSE Ratio: 1つ先のtimeの予測結果と比較して、どれぐらいRMSEが悪化したかを測る指標
pred_step = 1, rmse = 6.752263505030609, rmse_ratio = 0.0
pred_step = 2, rmse = 7.427930799122002, rmse_ratio = 0.10006530307770192
pred_step = 3, rmse = 8.084223281726013, rmse_ratio = 0.19726122591380807
pred_step = 4, rmse = 9

In [ ]:
# 12時間後を予測しているため、予測していている時間を取り出す
test_times12 = test_times.reshape(-1)+ np.timedelta64(1, 'D')
# 最後の時間は正解データがないため、削除
test_times12 = np.delete(test_times12, -1)

# 12時間後を予測している行以外を削除
preds12 = preds[11, :]
# # 最後の時間は正解データがないため、削除
preds12 = np.delete(preds12, -1)

# 正解データの最初の行を削除
test_labels12 = np.delete(test_labels.reshape(-1), -1)

# 出力用ファイル
# preds_file = np.concatenate([test_times12, test_labels12, preds12])

# 予測結果dataframe作成
# df_preds12 = pd.DataFrame(data = [[test_times12, test_labels12, preds12]], columns=["Date", "Label", "Pred"])
df_preds12 = pd.DataFrame({'Date': test_times12, 'Label': test_labels12, 'Pred': preds12})

# 保存
df_preds12.to_csv('/content/drive/MyDrive/python/02_電力売買予測/preds.csv')
df_preds12

,Date,Label,Pred
0,2022-05-03 03:00:00,20.76,18.103525
1,2022-05-03 04:00:00,19.76,18.131145
2,2022-05-03 05:00:00,20.98,18.158733
3,2022-05-03 06:00:00,15.82,18.177790
4,2022-05-03 07:00:00,4.65,18.177832
...,...,...,...
3400,2022-09-21 19:00:00,23.44,7.794650
3401,2022-09-21 20:00:00,20.48,7.794720
3402,2022-09-21 21:00:00,19.28,7.794735
3403,2022-09-21 22:00:00,11.77,7.794735


# 考察
## 予測データの妥当性
本予測データをそのまま利用することはできないレベルであると考える。RMSEが10前後と大きく、また、今回は提出しないが、予測でのその日の最高値、最安値と実際の最高値、最安値があっていない。出力したExcelデータを直接確認しても、予測値と正解値の乖離が大きい。まずはその日の最高値、最安値を高い精度で予測できるよう、モデルを改良する必要がある。  

## モデルの改良案
### LSTMの多層化
モデルを直接変更する場合は、表現力を上げるためにLSTMを多層化するとよいと考える。しかし、LSTMを3層、5層にした場合、計算時間が大幅に向上するにもかかわらず、TrainでのLossが80以上、TestのLossが140以上と精度が大幅に低下した。

### 入力データの標準化
気温等を標準化することで、予測精度が向上すると考える。

### 欠損値の少ないデータを選択する
天気データでは、変数によっては4000個ほどの欠損値が存在した。データ取得時点で、欠損値が少なくなるよう、取得先を考える必要がある。

### データ分析
電力売買価格のヒストグラムの確認や、データ間の相関を確認し、適切に処理を行うことで、予測精度が向上する可能性がある。


In [ ]:
test_times12.shape

(3405,)

In [ ]:
preds12.shape

(3405,)

In [ ]:
test_labels12.shape

(3405,)

# 検証用

In [ ]:
df = pd.read_excel(data_path)
df

,Date,Temp,SunshineDuration,Atm,Rainfall,Weather,Humidity,Visibility,Price
0,2017-09-21 01:00:00,21.6,0.0,1008.6,0.0,4.0,86.0,10.20,4.62
1,2017-09-21 02:00:00,20.9,0.0,1008.6,0.0,4.0,90.0,9.13,4.59
2,2017-09-21 03:00:00,20.7,0.0,1008.6,0.0,4.0,92.0,7.17,5.04
3,2017-09-21 04:00:00,20.8,0.0,1008.7,0.0,4.0,93.0,6.98,5.04
4,2017-09-21 05:00:00,20.8,0.0,1009.0,0.0,4.0,92.0,7.03,5.04
...,...,...,...,...,...,...,...,...,...
35059,2022-09-20 20:00:00,18.0,0.0,1008.7,0.0,2.0,59.0,20.00,20.48
35060,2022-09-20 21:00:00,17.7,0.0,1009.3,0.0,2.0,63.0,20.00,19.28
35061,2022-09-20 22:00:00,17.5,0.0,1009.6,0.0,4.0,67.0,20.00,11.77
35062,2022-09-20 23:00:00,17.6,0.0,1009.6,0.0,4.0,68.0,20.00,7.42


In [ ]:
pd.DataFrame(dataset_inputs[:, 0, :])

,0,1,2,3,4,5,6,7
0,21.6,0.0,1008.6,0.0,4.0,86.0,10.20,4.62
1,20.9,0.0,1008.6,0.0,4.0,90.0,9.13,4.59
2,20.7,0.0,1008.6,0.0,4.0,92.0,7.17,5.04
3,20.8,0.0,1008.7,0.0,4.0,93.0,6.98,5.04
4,20.8,0.0,1009.0,0.0,4.0,92.0,7.03,5.04
...,...,...,...,...,...,...,...,...
34051,30.4,0.0,1006.1,0.0,4.0,67.0,20.00,35.00
34052,29.6,0.0,1006.6,0.0,4.0,74.0,20.00,23.69
34053,29.0,0.0,1006.7,0.0,4.0,72.0,20.00,17.55
34054,28.0,0.0,1006.6,0.0,2.0,77.0,20.00,10.00


In [ ]:
pd.DataFrame(dataset_labels)

,0
0,5.63
1,6.84
2,7.55
3,7.69
4,7.62
...,...
34051,20.48
34052,19.28
34053,11.77
34054,7.42


In [ ]:
pd.DataFrame(dataset_times)

,0
0,2017-11-02 01:00:00
1,2017-11-02 02:00:00
2,2017-11-02 03:00:00
3,2017-11-02 04:00:00
4,2017-11-02 05:00:00
...,...
34051,2022-09-20 20:00:00
34052,2022-09-20 21:00:00
34053,2022-09-20 22:00:00
34054,2022-09-20 23:00:00


In [ ]:
dataset_times

array([['2017-11-02T01:00:00.000000000'],
       ['2017-11-02T02:00:00.000000000'],
       ['2017-11-02T03:00:00.000000000'],
       ...,
       ['2022-09-20T22:00:00.000000000'],
       ['2022-09-20T23:00:00.000000000'],
       ['2022-09-21T00:00:00.000000000']], dtype='datetime64[ns]')

In [ ]:
test = dataset_times + np.timedelta64(1, 'D')
test

array([['2017-11-03T01:00:00.000000000'],
       ['2017-11-03T02:00:00.000000000'],
       ['2017-11-03T03:00:00.000000000'],
       ...,
       ['2022-09-21T22:00:00.000000000'],
       ['2022-09-21T23:00:00.000000000'],
       ['2022-09-22T00:00:00.000000000']], dtype='datetime64[ns]')

In [ ]:
test2 = dataset_times.reshape(-1)
test2

array(['2017-11-02T01:00:00.000000000', '2017-11-02T02:00:00.000000000',
       '2017-11-02T03:00:00.000000000', ...,
       '2022-09-20T22:00:00.000000000', '2022-09-20T23:00:00.000000000',
       '2022-09-21T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
test3 = ["a", "b", "c", "d"]
df = pd.DataFrame(
    data=np.random.randint(
        0, 10, (6,4)),
    columns = test3)
print(df)

   a  b  c  d
0  1  4  5  2
1  3  1  3  4
2  2  2  4  1
3  0  3  8  1
4  4  7  5  7
5  1  0  0  1


In [ ]:
test_labels.reshape(-1).shape

(3406,)

In [ ]:
print(test2.shape)
test4 = np.delete(test2, -1)
test4.shape

(34056,)


(34055,)